# Code to functions

One of the nice things about Python (and Jupyter notebooks) is that we can write functions 'on the fly' and apply them to our data.  This can simply the process of repetitive coding, give us clearer and more readable code, improve reprodcibility, and allow us to make up for the shortcomings of built-in functions. 

This is an example from [pyOpenSci](https://github.com/pyOpenSci/) that shows the potential benefits of writing your own functions.  We're going to import using Pandas a simple CSV file that has a list of locations, columns with each location's average snowfall and temperature, and the country or geography region where the location can be found.  We'll then do some simple operations to group locations by their snowfall and look at their average temperature:

In [ ]:
# import Pandas
import pandas as pd

# Load the data 
data_path = "snow_data.csv"
data = pd.read_csv(data_path)
print(data.head())


In [ ]:
# Filter rows where the value in 'column1', the average snowfall, is greater than 10, and assign to a new DataFrame
sites_more_snow = data[data["average_snowfall"] > 15]

# we can print this result now
print(f"Sites with more than 15 inches of snow: \n{sites_more_snow}")


In [ ]:
# We can then use the .mean method to get the average temperature for those sites 
avg_temp_more_snow = sites_more_snow["average_temperature"].mean()
print(f"The mean temp for sites with more snow is {avg_temp_more_snow}")


In [ ]:
# Now let's find sites with a small accumulation of snow on average and once again calculate the average temperature for those sites
sites_less_snow = data[data["average_snowfall"] < 5]
avg_temp_less_snow = sites_less_snow["average_temperature"].mean()

print(f"The mean temp for sites with less snow is {avg_temp_less_snow}")


In [ ]:
# Finally, let's filter rows where the average snowfall is between 5 and 15 inches
sites_medium_snow = data[(data["average_snowfall"] >= 5) & (data["average_snowfall"] <= 15)]

print(f"Sites with snowfall between 5 and 15 inches:\n{sites_medium_snow}")


In [ ]:
# And calculate the mean temperature for sites with medium snow
avg_temp_medium_snow = sites_medium_snow["average_temperature"].mean()
print(
    f"The mean temperature for sites with medium snow is {avg_temp_medium_snow}"
)

## Is there a 'cleaner' way?

That worked, but it was repetitive and a little mind numbing.  We did a few different things over and over again - we filtered the list by average snowfall, saved those rows as a new DataFrame, and then calculated the average temperature for those sites.  

Here's an alternative approach using a set of simple functions to do the repetitive work for us.  Once again, this is from [pyOpenSci](https://github.com/pyOpenSci/):

In [7]:
# create a function that reads in the data and replaces any missing data
def load_data(filepath):

    # Load the data and replace the missing value flag of -999 with NaN
    return pd.read_csv(filepath, na_values=-999) # this is a Pandas function to read CSV data and deal with missing value flags


# create a function that catagorizes a site by the amount of snow
def categorize_snow(x):
    if x > 15:
        return "High"
    elif x < 5:
        return "Low"
    else:
        return "Medium"

# create a function that adds a column to our DataFrame with the snow catagory
def add_snowfall_category(data):
    data["snowfall_category"] = data["average_snowfall"].apply(categorize_snow) # note that this applies the function above this one to the data! 
    return data

# Create a summary DataFrame that shows the characteristics of each group of locations in each snow category
def summarize_data(data):
    summary = data.groupby("snowfall_category").agg(
        avg_snowfall=("average_snowfall", "mean"),
        avg_temperature=("average_temperature", "mean"),
        site_count=("site", "count"),
    )
    return summary


When we run the code cell above, those functions are now available to us in this session! In the code block below, we'll call those functions to do the work for us:

In [ ]:
# Load the data and deal with any missing data using the first function we wrote above
data = load_data(data_path)
data


In [ ]:
# Add snowfall category to our DataFrame using the third function we wrote above, which calls the second function we wrote to do the categorization
data = add_snowfall_category(data)
data


In [ ]:
# Summarize the data using the fourth function we wrote
summary = summarize_data(data)
summary


## Creating and using your own module

Finally, if you find yourself using functions you wrote on a regular basis, you might want to put them in a `.py` file you can import just like we import `Numpy` or `Pandas`.  Once again based on the example provided by example from [pyOpenSci](https://github.com/pyOpenSci/), the functions we wrote above are all stored in a single Python file called `my_module.py`

In [ ]:
from my_module import load_data, summarize_data, categorize_snowfall_amount

data_path = "snow_data.csv"

# Load and clean the data
data = load_data(data_path)

# Add snowfall category
data = categorize_snowfall_amount(data)

# Summarize the data
summary = summarize_data(data)
summary